In [1]:
import random
import re
import pickle

import openai

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.l2 import Ether
from scapy.layers.inet import IP, TCP, ICMP

E:\anaconda\envs\PAC-GPT\lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  cipher=algorithms.Blowfish,
E:\anaconda\envs\PAC-GPT\lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated and will be removed in a future release
  cipher=algorithms.CAST5,


In [7]:
print(openai.__version__)
# （删去）111sk-proj-5LvxYFhISscPhn6z90J2T3BlbkFJMReVCAyueMzVGYM0mfsv111（删去）
openai.api_key = 'api-key'

0.28.0


In [11]:
# Set the proxy URL and port
proxy_url = 'http://127.0.0.1'
proxy_port = '7890'

# Set the http_proxy and https_proxy environment variables
os.environ['http_proxy'] = f'{proxy_url}:{proxy_port}'
os.environ['https_proxy'] = f'{proxy_url}:{proxy_port}'

In [3]:
packets_per_request = 5

In [4]:
with open("../data/text/three_summaries.txt","r", encoding="utf-8") as f:
    packets_summary = f.read().splitlines()

In [5]:
len(packets_summary)

9999

In [12]:
engines = openai.Engine.list()
print(engines)

{
  "object": "list",
  "data": [
    {
      "object": "engine",
      "id": "dall-e-3",
      "ready": true,
      "owner": "system",
      "permissions": null,
      "created": null
    },
    {
      "object": "engine",
      "id": "dall-e-2",
      "ready": true,
      "owner": "system",
      "permissions": null,
      "created": null
    },
    {
      "object": "engine",
      "id": "tts-1-hd-1106",
      "ready": true,
      "owner": "system",
      "permissions": null,
      "created": null
    },
    {
      "object": "engine",
      "id": "tts-1-hd",
      "ready": true,
      "owner": "system",
      "permissions": null,
      "created": null
    },
    {
      "object": "engine",
      "id": "babbage-002",
      "ready": true,
      "owner": "system",
      "permissions": null,
      "created": null
    },
    {
      "object": "engine",
      "id": "text-embedding-3-small",
      "ready": true,
      "owner": "system",
      "permissions": null,
      "created": null
   

In [13]:
responses = pickle.load( open("../data/pickle/davinci_pairs2.pkl", "rb" ))

In [14]:
len(responses)

200

In [15]:
for i in range(1):
    summaries = random.choices(packets_summary,k=packets_per_request)
    query = "This is a packet summary:\n"
    query += "192.168.1.194 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x9015, seq=18917\n"
    query += "This is the python code to generate the packet:\n"
    query += 'scapy.IP(src="192.168.1.194", dst="192.168.1.190")/scapy.ICMP(type=8, id=0x9015, seq=18917)\n\n'
    query += "This is a packet summary:\n"
    query += "192.168.1.190 → 192.168.1.152 DNS 193 Standard query response 0x0971 No such name AAAA a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com SOA ns-1926.awsdns-48.co.uk\n"
    query += "This is the python code to generate the packet:\n"
    query += 'IP(src="192.168.1.190", dst="192.168.1.152")/DNS(id=0x0971, qr=1, opcode=0, rcode=3, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com", qtype="SOA", qclass="IN"))\n\n'

    query += "These are the packet summaries:\n"
    text_sum = ""
    for summary in summaries:
        query += summary + "\n"
        text_sum += summary + "\n"

    query += "\nGenerate python code for creating these packets with scapy framework and put them all in a list named pkt_list."

    print(query)
    break
    
    completion = openai.Completion.create(engine="text-davinci-003", prompt=query,max_tokens=2600,temperature=0.1)
    completion["prompt_summary"] = text_sum
    responses.append(completion)
    # responses即text-davinci-003生成的scapy代码
    #print(responses[-1].choices[0].text)
    
    exec(responses[-1].choices[0].text)
    # pickle用于序列化数据，文件格式是pkl
    pickle.dump(responses, open( "../data/pickle/davinci_pairs2.pkl", "wb" ) )
    print(len(responses))


This is a packet summary:
192.168.1.194 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x9015, seq=18917
This is the python code to generate the packet:
scapy.IP(src="192.168.1.194", dst="192.168.1.190")/scapy.ICMP(type=8, id=0x9015, seq=18917)

This is a packet summary:
192.168.1.190 → 192.168.1.152 DNS 193 Standard query response 0x0971 No such name AAAA a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com SOA ns-1926.awsdns-48.co.uk
This is the python code to generate the packet:
IP(src="192.168.1.190", dst="192.168.1.152")/DNS(id=0x0971, qr=1, opcode=0, rcode=3, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com", qtype="SOA", qclass="IN"))

These are the packet summaries:
192.168.1.194 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x9015, seq=19032
192.168.1.152 → 192.168.1.190 DNS 109 Standard query 0x7184 A a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com
192.168.1.190 → 192.168.1.192 ICMP 100 Echo (ping) reply id=0x112d, seq

In [17]:
print(responses[-1].choices[0].text)



pkt_list = [scapy.IP(src="192.168.1.192", dst="192.168.1.190")/scapy.ICMP(type=8, id=0x112d, seq=6152),
            scapy.IP(src="192.168.1.190", dst="192.168.1.193")/scapy.ICMP(type=0, id=0x0001, seq=4602),
            scapy.IP(src="192.168.1.190", dst="192.168.1.192")/scapy.ICMP(type=0, id=0x112d, seq=6295),
            scapy.IP(src="192.168.1.152", dst="192.168.1.190")/DNS(id=0x7e4a, qr=0, opcode=0, rcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com", qtype="AAAA", qclass="IN")),
            scapy.IP(src="192.168.1.190", dst="192.168.1.194")/scapy.ICMP(type=0, id=0x9015, seq=18619)]


In [16]:
exec(responses[-1].choices[0].text)
print(pkt_list)

[<IP  frag=0 proto=icmp src=192.168.1.192 dst=192.168.1.190 |<ICMP  type=echo-request id=0x112d seq=0x1808 |>>, <IP  frag=0 proto=icmp src=192.168.1.190 dst=192.168.1.193 |<ICMP  type=echo-reply id=0x1 seq=0x11fa |>>, <IP  frag=0 proto=icmp src=192.168.1.190 dst=192.168.1.192 |<ICMP  type=echo-reply id=0x112d seq=0x1897 |>>, <IP  src=192.168.1.152 dst=192.168.1.190 |<DNS  id=32330 qr=0 opcode=QUERY rcode=ok qdcount=1 ancount=0 nscount=0 arcount=0 qd=<DNSQR  qname='a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com' qtype=AAAA qclass=IN |> |>>, <IP  frag=0 proto=icmp src=192.168.1.190 dst=192.168.1.194 |<ICMP  type=echo-reply id=0x9015 seq=0x48bb |>>]


In [18]:
print(responses[-1].prompt_summary)
print(responses[-1].choices[0].text)

192.168.1.192 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x112d, seq=6152
192.168.1.190 → 192.168.1.193 ICMP 76 Echo (ping) reply id=0x0001, seq=4602
192.168.1.190 → 192.168.1.192 ICMP 100 Echo (ping) reply id=0x112d, seq=6295
192.168.1.152 → 192.168.1.190 DNS 109 Standard query 0x7e4a AAAA a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com
192.168.1.190 → 192.168.1.194 ICMP 100 Echo (ping) reply id=0x9015, seq=18619



pkt_list = [scapy.IP(src="192.168.1.192", dst="192.168.1.190")/scapy.ICMP(type=8, id=0x112d, seq=6152),
            scapy.IP(src="192.168.1.190", dst="192.168.1.193")/scapy.ICMP(type=0, id=0x0001, seq=4602),
            scapy.IP(src="192.168.1.190", dst="192.168.1.192")/scapy.ICMP(type=0, id=0x112d, seq=6295),
            scapy.IP(src="192.168.1.152", dst="192.168.1.190")/DNS(id=0x7e4a, qr=0, opcode=0, rcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com", qtype="AAAA", qclass="IN")),
            scapy.IP

In [19]:
pickle.dump(responses, open( "../data/pickle/davinci_pairs2.pkl", "wb" ) )

In [20]:
fixed_text = """

pkt_list = [scapy.IP(src="192.168.1.152", dst="192.168.1.190")/scapy.DNS(id=0x8d29, qr=0, opcode=0, rcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com", qtype="A", qclass="IN")),
            scapy.IP(src="192.168.1.193", dst="192.168.1.190")/scapy.ICMP(type=8, id=0x0001, seq=5456),
            scapy.IP(src="192.168.1.190", dst="192.168.1.192")/scapy.ICMP(type=0, id=0x112d, seq=5469),
            scapy.IP(src="192.168.1.190", dst="192.168.1.192")/scapy.ICMP(type=0, id=0x112d, seq=6133),
            scapy.IP(src="192.168.1.190", dst="192.168.1.194")/scapy.ICMP(type=0, id=0x9015, seq=19453)]
"""



In [21]:
exec(fixed_text)

In [22]:
pkt_list

[<IP  src=192.168.1.152 dst=192.168.1.190 |<DNS  id=36137 qr=0 opcode=QUERY rcode=ok qdcount=1 ancount=0 nscount=0 arcount=0 qd=<DNSQR  qname='a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com' qtype=A qclass=IN |> |>>,
 <IP  frag=0 proto=icmp src=192.168.1.193 dst=192.168.1.190 |<ICMP  type=echo-request id=0x1 seq=0x1550 |>>,
 <IP  frag=0 proto=icmp src=192.168.1.190 dst=192.168.1.192 |<ICMP  type=echo-reply id=0x112d seq=0x155d |>>,
 <IP  frag=0 proto=icmp src=192.168.1.190 dst=192.168.1.192 |<ICMP  type=echo-reply id=0x112d seq=0x17f5 |>>,
 <IP  frag=0 proto=icmp src=192.168.1.190 dst=192.168.1.194 |<ICMP  type=echo-reply id=0x9015 seq=0x4bfd |>>]

In [23]:
responses[-1].choices[0].text = fixed_text

In [24]:
print(responses[-1].prompt_summary)
print(responses[-1].choices[0].text)

192.168.1.192 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x112d, seq=6152
192.168.1.190 → 192.168.1.193 ICMP 76 Echo (ping) reply id=0x0001, seq=4602
192.168.1.190 → 192.168.1.192 ICMP 100 Echo (ping) reply id=0x112d, seq=6295
192.168.1.152 → 192.168.1.190 DNS 109 Standard query 0x7e4a AAAA a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com
192.168.1.190 → 192.168.1.194 ICMP 100 Echo (ping) reply id=0x9015, seq=18619



pkt_list = [scapy.IP(src="192.168.1.152", dst="192.168.1.190")/scapy.DNS(id=0x8d29, qr=0, opcode=0, rcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com", qtype="A", qclass="IN")),
            scapy.IP(src="192.168.1.193", dst="192.168.1.190")/scapy.ICMP(type=8, id=0x0001, seq=5456),
            scapy.IP(src="192.168.1.190", dst="192.168.1.192")/scapy.ICMP(type=0, id=0x112d, seq=5469),
            scapy.IP(src="192.168.1.190", dst="192.168.1.192")/scapy.ICMP(type=0, id=0x112d, seq=6133),
            scapy

In [25]:
del responses[-1]

In [26]:

pkt_list = []

pkt_list.append(scapy.IP(src="192.168.1.190", dst="192.168.1.193")/scapy.ICMP(type=76, id=0x0001, seq=4091))

pkt_list.append(scapy.IP(src="192.168.1.194", dst="192.168.1.190")/scapy.ICMP(type=100, id=0x9015, seq=18476))

pkt_list.append(scapy.IP(src="192.168.1.192", dst="192.168.1.190")/scapy.ICMP(type=100, id=0x112d, seq=5799))

pkt_list.append(scapy.IP(src="192.168.1.194", dst="192.168.1.190")/scapy.ICMP(type=100, id=0x9015, seq=19355))

pkt_list.append(scapy.IP(src="192.168.1.193", dst="192.168.1.190")/scapy.ICMP(type=76, id=0x0001, seq=4322))

pkt_list.append(scapy.IP(src="192.168.1.192", dst="192.168.1.190")/scapy.ICMP(type=100, id=0x112d, seq=5804))

pkt_list.append(scapy.IP(src="192.168.1.190", dst="192.168.1.194")/scapy.ICMP(type=100, id=0x9015, seq=19228))

pkt_list.append(scapy.IP(src="192.168.1.190", dst="192.168.1.194")/scapy.ICMP(type=100, id=0x9015, seq=19718))

pkt_list.append(scapy.IP(src="192.168.1.193", dst="192.168.1.190")/scapy.ICMP(type=76, id=0x0001, seq=5336))

In [27]:
with open("../data/pcapgenerated_curie_commands.pcap", "wb") as f:
    wrpcap(f, pkt_list)

In [28]:
print(responses[0])

{
  "id": "cmpl-6oqdZkXJZj4oXycbtlpbzYMv4Jzfz",
  "object": "text_completion",
  "created": 1677576757,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\n\npkt_list = [scapy.IP(src=\"192.168.1.192\", dst=\"192.168.1.190\")/scapy.ICMP(type=8, id=0x112d, seq=5619),\n            scapy.IP(src=\"192.168.1.194\", dst=\"192.168.1.190\")/scapy.ICMP(type=8, id=0x9015, seq=19673),\n            scapy.IP(src=\"192.168.1.194\", dst=\"192.168.1.190\")/scapy.ICMP(type=8, id=0x9015, seq=18836),\n            scapy.IP(src=\"192.168.1.190\", dst=\"192.168.1.193\")/scapy.ICMP(type=0, id=0x0001, seq=4886),\n            scapy.IP(src=\"192.168.1.190\", dst=\"192.168.1.193\")/scapy.ICMP(type=0, id=0x0001, seq=4305)]",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 519,
    "completion_tokens": 254,
    "total_tokens": 773
  },
  "prompt_summary": "192.168.1.192 \u2192 192.168.1.190 ICMP 100 Echo (ping) request id=0x112d, 